In [1]:
!pip install -q opencv-python
!pip install -q gradio
!pip install -q scipy
!pip install -q dlib
!pip install -q gradio opencv-python pygame

In [2]:
import gradio as gr
import numpy as np
import cv2
import pygame
import dlib
from gradio import Image
import numpy as np
from scipy.spatial import distance as dist

pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
sound_path= "alarm1.mp3"    #"/content/drive/MyDrive/preview.mp3"
def play_sound():
    """Plays a sound using pygame."""
    pygame.mixer.init()
    sound = pygame.mixer.Sound(sound_path)
    sound.play()
    return "Sound played!"  # Return a message to display


In [5]:
import cv2
import numpy as np
# from google.colab.patches import cv2_imshow
def outline_face(frame):
    # Load Haar cascade classifiers
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    eye_cascade = cv2.CascadeClassifier("haarcascade_eye.xml")
    mouth_cascade = cv2.CascadeClassifier("haarcascade_mcs_mouth.xml")
    #smile_cascade = cv2.CascadeClassifier("haarcascade_smile.xml")
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Draw rectangles around faces and eyes
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)  # Face rectangle (blue)
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]
        muts = mouth_cascade.detectMultiScale(roi_gray)
        #smiles = smile_cascade.detectMultiScale(roi_gray)
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (mx, my, mw, mh) in muts:
            if mw > 0 and mh > 0:
                cv2.rectangle(roi_color, (mx, my), (mx + mw, my + mh), (0, 255, 0), 2)   
        for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)  # Eye rectangle (green)
    return frame

In [6]:
def improve_lighting(frame, use_clahe=False, clip_limit=2.0, tile_grid_size=(8, 8), brightness_threshold=100):

    # Convert the frame to YCrCb color space
    ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    
    # Compute average brightness (mean of the Y channel)
    avg_brightness = np.mean(ycrcb[:, :, 0])
    
    if avg_brightness < brightness_threshold:
        # Apply enhancement only if brightness is below the threshold
        if use_clahe:
            # Apply CLAHE on the Y channel
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
            ycrcb[:, :, 0] = clahe.apply(ycrcb[:, :, 0])
        else:
            # Apply standard histogram equalization on the Y channel
            ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
    
    # Convert back to BGR color space
    enhanced_frame = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)
    return enhanced_frame


In [7]:
def compute_ear(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

In [8]:
# Class to encapsulate drowsiness detection logic
class DrowsinessDetector:
    def __init__(self, ear_threshold=0.22, consec_frames=20, shape_predictor_path="shape_predictor_68_face_landmarks.dat"):
        self.ear_threshold = ear_threshold
        self.consec_frames = consec_frames
        self.counter = 0
        self.alarm_on = False

        # Load dlib's models
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(shape_predictor_path)

        # Define eye landmarks
        self.LEFT_EYE_POINTS = list(range(42, 48))
        self.RIGHT_EYE_POINTS = list(range(36, 42))

    def detect_drowsiness(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.detector(gray)
        
        for face in faces:
            # Get landmarks and extract eye regions
            shape = self.predictor(gray, face)
            landmarks = np.array([[p.x, p.y] for p in shape.parts()])
            
            # Extract left and right eye coordinates
            try:
                left_eye = landmarks[self.LEFT_EYE_POINTS]
                right_eye = landmarks[self.RIGHT_EYE_POINTS]
            except IndexError:
                # Handle case where eyes are not detected correctly
                print("Error: Eye landmarks not detected properly.")
                return frame  # Return the frame without processing further

            # Compute EAR (Eye Aspect Ratio) for both eyes
            left_ear = compute_ear(left_eye) if left_eye is not None else 0.0
            right_ear = compute_ear(right_eye) if right_eye is not None else 0.0
    
            # Handle case where either left or right EAR is missing
            if left_ear == 0.0 or right_ear == 0.0:
                print("Warning: One or both eyes are not visible.")
                ear = 0.0  # Set EAR to a default value when eyes are not visible
            else:
                # Compute average EAR
                ear = (left_ear + right_ear) / 2.0

            # Check EAR threshold for drowsiness detection
            if ear < self.ear_threshold:
                self.counter += 1
                if self.counter >= self.consec_frames and not self.alarm_on:
                    print("Drowsiness detected!")
                    self.alarm_on = True
                    self.trigger_alarm()
            else:
                self.counter = 0
                self.alarm_on = False

            # Visual feedback: Drawing eye landmarks and EAR value on the frame
            if left_eye is not None:
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 1)
            if right_eye is not None:
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 1)
                cv2.putText(frame, f"Alertness: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
        return frame

    def trigger_alarm(self):
        play_sound()


In [9]:
def sleep_detections(frame, detector):
    return detector.detect_drowsiness(frame)

In [10]:
def centralize_face(frame,resize_dim=(600, 600)):
    """
    Crop and centralize the face in the frame.
    """
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    image_padding= 20
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces) > 0:
        # Take the first detected face
        x, y, w, h = faces[0]
        # Ensure coordinates are within bounds
        x = max(0, x-image_padding)
        y = max(0, y-image_padding)
        w = min(w+image_padding, frame.shape[1] - x)
        h = min(h+image_padding, frame.shape[0] - y)
        # Crop the face
        face = frame[y:y+h, x:x+w]
        # Resize the face to the specified dimensions
        central_frame = cv2.resize(face, resize_dim)
        return central_frame
    else:
        # Return the original frame if no face is detected
        return frame

In [11]:
def Image_Enhancement(frame):
    frame= improve_lighting(frame,use_clahe=True)
    frame= centralize_face(frame)
    return (frame) 

In [12]:
def facedection_cv2(frame, facedection):
    frame = frame.copy()
    if facedection == "Face Detection":
        frame = outline_face(frame)
    elif facedection == "Sleep Detection":
        frame = sleep_detections(frame,detector)
    elif facedection == "Image Enhancement":
        frame = Image_Enhancement(frame)
    return frame

In [13]:
image = gr.Image(value="logo.jpeg", width=150, height=150, show_label=False)
detector = DrowsinessDetector()
with gr.Blocks(
    title="Driver Drowsiness App",
    css="""
    .dialog-box {border: 1px solid #ccc; border-radius: 8px; padding: 20px; background-color:white;} 
    .header-box { border: 1px solid #ccc; background-color: white; padding: 10px; text-align: center;} 
    """
) as demo:
    with gr.Group(elem_classes="dialog-box"):
        with gr.Row(elem_classes="header-box"):
            gr.Markdown("""  
            # Group 9 Driver Drowsiness
            ## Aditya, Ranjeet, and Rajesh  
            **Powered by Computer Vision**
            """)
            image.render()

    with gr.Row():
        with gr.Column():
            facedection = gr.Dropdown(
                choices=["Face Detection", "Sleep Detection", "Image Enhancement"],
                value="Face Detection",
                label="Select Detection Mode"
            )
            input_img = gr.Image(sources=["webcam"], type="numpy", label="Input Image")
        
        with gr.Column():
            output_img = gr.Image(streaming=True, label="Output Image",show_label=False)
        
        # Connect input to processing
        input_img.stream(facedection_cv2, [input_img, facedection], [output_img],
                         time_limit=30, stream_every=0.1, concurrency_limit=30)

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f5cd3d1adec8f93d17.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
Drowsiness detected!
